In [44]:
#!/usr/bin/python

import sys, os, re, numpy as np
from pyteomics import mzxml

def getParams(paramFile):
    params = dict()
    with open(paramFile, 'r') as file:
        for line in file:
            if re.search(r'^#', line) or re.search(r'^\s', line):
                continue
            line = re.sub(r'#.*', '', line) # Remove comments (start from '#')
            line = re.sub(r'\s*', '', line) # Remove all whitespaces
            key = line.split('=')[0]
            val = line.split('=')[1]
            params[key] = val
    return params


def detectPeaks(spec, params):
    # Parameters
    if params['data_acquisition_mode'] == "1":
        isCentroided = 1
    elif params['data_acquisition_mode'] == "2":
        isCentroided = 0
    else:
        print("Please set the proper 'data_acquisition_mode' parameter")
        sys.exit("")
    intensityThreshold = 0  # May come from a parameter file

    # m/z and intensity arrays from a spectrum object
    mzArray = spec['m/z array']
    intensityArray = spec['intensity array']
    nPeaks = len(mzArray)
    newMzArray = []
    newIntensityArray = []

    # Detect peaks (i.e. centroidization of MS1 spectrum)
    if isCentroided == 0:  # i.e. Profile mode MS1
        for i in range(2, nPeaks - 1):
            if intensityArray[i] > 0:
                # Consider 2 points before and after the point of interest x, i.e. 5 point window
                b2, b1, x, a1, a2 = intensityArray[(i - 2):(i + 3)]
                if x >= intensityThreshold:
                    if isMax(b2, b1, x, a1, a2):
                        # If x is the local maximum in a 5-point window, lower and upper bounds for a peak will be explored
                        # Refer Figure 1a and b in the paper, Cox and Mann, Nature Biotech. 2008; 26: 1367-22
                        minInd = findMinPeakIndex(i, intensityArray)
                        maxInd = findMaxPeakIndex(i, intensityArray)
                        if (maxInd - minInd) > 2:
                            newMz, newIntensity = findPeakCenter(minInd, i, maxInd, mzArray, intensityArray)
                            newMzArray.append(newMz)
                            newIntensityArray.append(newIntensity)

        # Update 'spec' object
        spec['m/z array'] = newMzArray
        spec['intensity array'] = newIntensityArray

    # Do nothing for centroid mode MS1
    return spec


def isMax(b2, b1, x, a1, a2):
    if x > b1 and x > a1:
        return True
    if x > b2 and x == b1 and x > a1:
        return True
    if x > b1 and x == a1 and x > a2:
        return True
    return False


def findMinPeakIndex(ind, array):
    while ind > 0 and array[ind] != 0 and array[ind - 1] <= array[ind]:
        ind -= 1
    return ind + 1


def findMaxPeakIndex(ind, array):
    count = len(array)
    while ind < count and array[ind] != 0 and array[ind + 1] <= array[ind]:
        ind += 1
    return ind - 1


def findPeakCenter(minInd, centerInd, maxInd, mz, intensity):
    # Find the center of a peak composed of five data points
    centerMz = 0
    centerIntensity = 0
    for i in range(minInd, maxInd + 1):
        if intensity[i] >= centerIntensity:
            centerIntensity = intensity[i]  # Take the maximum intensity as a peak intensity

    # There's a plateau, bu others are zeros
    if minInd == maxInd:
        centerMz = mz[maxInd]
        return centerMz, centerIntensity

    # Right-angled triangle-shaped peak
    if minInd == centerInd:
        centerMz = estimate2(mz[centerInd], mz[centerInd + 1], intensity[centerInd], intensity[centerInd + 1])
        return centerMz, centerIntensity

    # Left-angled triangle-shaped peak
    if maxInd == centerInd:
        centerMz = estimate2(mz[centerInd - 1], mz[centerInd], intensity[centerInd - 1], intensity[centerInd])
        return centerMz, centerIntensity

    # Typical bell(triangle)-shaped peak
    centerMz = estimate3(mz[centerInd - 1], mz[centerInd], mz[centerInd + 1], intensity[centerInd - 1], intensity[centerInd], intensity[centerInd + 1])
    return centerMz, centerIntensity


def estimate2(m1, m2, i1, i2):
    centerVal = (m1 * i1 + m2 * i2) / (i1 + i2) # Intensity-weighted average of m/z
    return centerVal


def estimate3(m1, m2, m3, i1, i2, i3):
    l1 = np.log(i1)
    l2 = np.log(i2)
    l3 = np.log(i3)
    centerVal = 0.5 * ((l1 - l2) * (m3 ** 2 - m1 ** 2) - (l1 - l3) * (m2 ** 2 - m1 ** 2)) / ((l1 - l2) * (m3 - m1) - (l1 - l3) * (m2 - m1));
    return centerVal



In [45]:
# Input: mzXML file
# To-do: expand to mzML
inputFile = "/Research/Projects/7Metabolomics/JUMPm/IROAsamples/IROA_IS_NEG_1.mzXML"
reader = mzxml.MzXML(inputFile)

##############
# Parameters #
##############
paramFile = "jumpm_negative_desktop.params"
params = getParams(paramFile)
if params['data_acquisition_mode'] == "1":
    isCentroided = 1
elif params['data_acquisition_mode'] == "2":
    isCentroided = 0
else:
    print ("Please set the proper 'data_acquisition_mode' parameter")
    sys.exit("")
# firstScan = int(params['first_scan_extraction'])
# lastScan = int(params['last_scan_extraction'])
firstScan = 1000
lastScan = 1500
gap = params['skipping_scans']

# with reader:
#     ############################
#     # Get MS1 scan information #
#     ############################
#     nMS1 = 0
#     ms1ScanNumArray = []
#     for spec in reader:
#         msLevel = int(spec['msLevel'])
#         scanNum = int(spec['num'])
#         if msLevel == 1 and scanNum >= firstScan and scanNum <= lastScan:
#             nMS1 += 1
#             ms1ScanNumArray.append(spec['num'])
#         elif scanNum > lastScan:
#             break

#     for scan in ms1ScanNumArray:
#         scanbuf = getScan(scan, )

OSError: [Errno 22] Invalid argument